In [1]:
%load_ext autoreload
%autoreload 2
import torch
from torchvision import datasets, transforms
torch.manual_seed(42)
import json
import os
import os.path
from twosample import get_artifical_initial_clustering, utils
from twosample.affinity import fast_twosample_train_test_split, fast_twosample_train, fast_twosample_test
from twosample.architectures.resnet import resnet18
import torch.optim as optim
import numpy as np
import gc
from sklearn.cluster import AgglomerativeClustering

In [2]:
hp = {}
hp['dataset'] = 'imagenet-full'
hp['initial_clustering'] = {}
hp['initial_clustering']['num_clusters'] = 10000
hp['initial_clustering']['minimum_initial_cluster_size'] = 80
hp['num_merges_per_network_update'] = 3000
hp['desired_num_clusters'] = 1000
hp['epochs'] = 15
hp['affinity'] = {}
hp['affinity']['batch_size'] = 128
hp['affinity']['train_ratio'] = 0.5
hp['affinity']['log_interval'] = 200
hp['device'] = torch.device('cuda')
hp['learning_rate'] = 0.001
hp['weight_decay'] = 0
hp['sobel'] = False
hp['augmentation'] = {}
hp['augmentation'] = {'random_horizontal_flip': False,
                      'random_resized_crop': False}

In [3]:
def create_and_save_train_test_split(current_clustering, experiment_directory, hp):
    sample_indices_train, sample_indices_test = fast_twosample_train_test_split(current_clustering, hp)
    clustering_checkpoint_directory = experiment_directory + '/{}'.format(len(current_clustering))
    utils.create_directory_if_not_exists(clustering_checkpoint_directory)
    current_clustering_filename = clustering_checkpoint_directory + '/current_clustering.npy'
    print('Save train-test split to', current_clustering_filename)
    np.save(current_clustering_filename, [sample_indices_train, sample_indices_test])
    checkpoints_directory = clustering_checkpoint_directory + '/checkpoints'
    affinities_directory = clustering_checkpoint_directory + '/affinities'
    utils.create_directory_if_not_exists(checkpoints_directory)
    utils.create_directory_if_not_exists(affinities_directory)
    mean_arithmetic_means = []
    mean_arithmetic_means_filename = clustering_checkpoint_directory + '/mean_arithmetic_means.csv'
    print('Save mean arithmetic means to', mean_arithmetic_means_filename)
    np.savetxt(mean_arithmetic_means_filename, mean_arithmetic_means)
    return sample_indices_train, sample_indices_test, clustering_checkpoint_directory, checkpoints_directory,\
        affinities_directory, mean_arithmetic_means

def run_experiment(hp):
    device = hp['device']
    experiment_directory = 'results/real/{}'.format(hp['dataset'])
    utils.create_directory_if_not_exists(experiment_directory)
    
    print('Running experiment with hyperparameters:\n{}'.format(hp))
    result = {}
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    transform_chain_train = []
    if hp['augmentation']['random_resized_crop']:
        transform_chain_train += [transforms.RandomResizedCrop(224)]
    else:
        transform_chain_train += [transforms.Resize(256), transforms.CenterCrop(224)]
    if hp['augmentation']['random_horizontal_flip']:
        transform_chain_train += [transforms.RandomHorizontalFlip()]
    transform_chain_train =  transforms.Compose(transform_chain_train + [transforms.ToTensor(), normalize])
    transform_chain_test =  transforms.Compose([transforms.Resize(256),
                                                transforms.CenterCrop(224),
                                                transforms.ToTensor(),
                                                normalize])
    dataset = datasets.ImageFolder('./datasets/' + hp['dataset'])
    
    
    clustering_checkpoints = [int(clustering_checkpoint_directory)
                              for clustering_checkpoint_directory in os.listdir(experiment_directory)]
    
    # If the experiment is started from scratch load the initial clustering provided by DeepCluster
    if len(clustering_checkpoints) == 0:
        # Check if DeepCluster has already been executed to provide an initial clustering
        deepcluster_clustering_filename = experiment_directory + '/deepcluster_clustering.npy'
        if not os.path.isfile(deepcluster_clustering_filename):
            args = {}
            args['verbose'] = True
            args['data'] = 'datasets/' + hp['dataset']
            args['arch'] = 'vgg16'
            args['sobel'] = True
            args['workers'] = 4
            args['resume'] = 'deepcluster/pretrained/vgg16/checkpoint.pth.tar'
            args['batch'] = 64
            args['clustering_filename'] = deepcluster_clustering_filename
            args['nmb_cluster'] = hp['initial_clustering']['num_clusters']
            deepcluster.main(args)
        
        print('Load initial clustering from', deepcluster_clustering_filename)
        original_current_clustering = np.load(deepcluster_clustering_filename).tolist()

        # Filter out all small clusters
        current_clustering = []
        total_num_observations = num_filtered_observations = 0
        for sample_indices in original_current_clustering:
            if len(sample_indices) >= hp['initial_clustering']['minimum_initial_cluster_size']:
                current_clustering.append(sample_indices)
            else:
                num_filtered_observations += len(sample_indices)
            total_num_observations += len(sample_indices)
        current_clustering = [sample_indices for sample_indices in current_clustering
                              if len(sample_indices) >= hp['initial_clustering']['minimum_initial_cluster_size']]
        print(('Removed all clusters that have less than {} observations. '
               '{} clusters removed. {} observations removed.').format(
            hp['initial_clustering']['minimum_initial_cluster_size'],
            utils.pretty_accuracy(len(original_current_clustering) - len(current_clustering),
                                  len(original_current_clustering)),
            utils.pretty_accuracy(num_filtered_observations, total_num_observations)
        ))
        
        # Create train-test split for the initial clustering and store it into a file.
        sample_indices_train, sample_indices_test, clustering_checkpoint_directory,\
            checkpoints_directory, affinities_directory, mean_arithmetic_means =\
                create_and_save_train_test_split(current_clustering, experiment_directory, hp)
    else:
        # Load latest clustering checkpoint
        clustering_checkpoint_directory = experiment_directory + '/{}'.format(
            min(clustering_checkpoints))
        current_clustering_filename = clustering_checkpoint_directory + '/current_clustering.npy'
        print('Load train-test split from', current_clustering_filename)
        sample_indices_train, sample_indices_test = np.load(current_clustering_filename).tolist()
        current_clustering = [a + b for a, b in zip(sample_indices_train, sample_indices_test)]
        mean_arithmetic_means_filename = clustering_checkpoint_directory + '/mean_arithmetic_means.csv'
        print('Load mean arithmetic means from', mean_arithmetic_means_filename)
        mean_arithmetic_means = np.loadtxt(mean_arithmetic_means_filename).tolist()
        checkpoints_directory = clustering_checkpoint_directory + '/checkpoints'
        affinities_directory = clustering_checkpoint_directory + '/affinities'
      
    while len(current_clustering) > hp['desired_num_clusters']:
        print('#Clusters:', len(sample_indices_train))

        model = resnet18(num_classes=len(sample_indices_train), sobel=hp['sobel'])
        model = model.to(device)
        optimizer = optim.Adam(filter(lambda x: x.requires_grad, model.parameters()),
                               weight_decay=hp['weight_decay'])
        loaded_epoch = 0

        # Check if their is a model checkpoint that can be loaded
        checkpoint_epochs = [int(checkpoint_filename.split('.')[0])
                             for checkpoint_filename in os.listdir(checkpoints_directory)]
        if len(checkpoint_epochs) > 0:
            loaded_epoch = max(checkpoint_epochs)
            checkpoint_filename = checkpoints_directory + '/{}.pth.tar'.format(loaded_epoch)
            print('Load model and optimizer state from', checkpoint_filename)
            checkpoint = torch.load(checkpoint_filename)
            model.load_state_dict(checkpoint['model'])
            optimizer.load_state_dict(checkpoint['optimizer'])

        optimizer.lr = hp['learning_rate']

        for epoch in range(loaded_epoch + 1, hp['epochs'] + 1):
            print('Epoch:', epoch, ', mean arithmetic means:', mean_arithmetic_means)
            dataset.transform = transform_chain_train
            train_loss = fast_twosample_train(sample_indices_train, dataset, hp, model, optimizer)

            checkpoint_filename = checkpoints_directory + '/{}.pth.tar'.format(epoch)
            print('Save model and optimizer state to', checkpoint_filename)
            torch.save({'model': model.state_dict(),
                        'optimizer' : optimizer.state_dict()},
                       checkpoint_filename)

            dataset.transform = transform_chain_test
            arithmetic_means = fast_twosample_test(sample_indices_test, dataset, hp, model)
            
            # Set diagonal to 0 for mean arithmetic mean calculation
            diagonal_indices = torch.arange(len(arithmetic_means), dtype=torch.long).to(device)
            arithmetic_means[diagonal_indices, diagonal_indices] = 0.
            mean_arithmetic_means.append(arithmetic_means.mean().item())
            mean_arithmetic_means_filename = clustering_checkpoint_directory + '/mean_arithmetic_means.csv'
            print('Save mean arithmetic means to', mean_arithmetic_means_filename)
            np.savetxt(mean_arithmetic_means_filename, mean_arithmetic_means)
            
            print('Epoch: {}, Train loss: {:.2f}, mean arithmetic mean: {:.10f}'.format(epoch, train_loss,
                                                                                        mean_arithmetic_means[-1]))
            # Set diagonal to 1
            arithmetic_means[diagonal_indices, diagonal_indices] = 1.

            affinities_filename = affinities_directory + '/{}.npy'.format(epoch)
            print('Save affinities to', affinities_filename)
            arithmetic_means = arithmetic_means.cpu().numpy()
            np.save(affinities_filename, arithmetic_means)
            
            # Check if mean arithmetic mean has decreased. If so load previous arithmetic means
            # Stop training and start merging immediately.
            early_stopping = epoch >= 2 and mean_arithmetic_means[-2] > mean_arithmetic_means[-1]
            if early_stopping:
                previous_affinities_filename = affinities_directory + '/{}.npy'.format(epoch - 1)
                print('Early stopping. Load previous affinities from', previous_affinities_filename)
                arithmetic_means = np.load(previous_affinities_filename)
                
            # If maximum epochs is reached or when early stopping is applicable perform a fixed
            # number of merges.
            if epoch == hp['epochs'] or early_stopping:
                desired_num_clusters = max(hp['desired_num_clusters'],
                                           len(current_clustering) - hp['num_merges_per_network_update'])
                print('Merging from {} clusters to {} clusters.'.format(len(current_clustering),
                                                                       desired_num_clusters))
                
                # Perform clustering with discriminability as affinity measure. Update the affinites
                # through average linkage.
                agglomerative_clustering = AgglomerativeClustering(linkage='average',
                                                                   affinity='precomputed',
                                                                   n_clusters=desired_num_clusters)
                agglomerative_clustering.fit(arithmetic_means)
                
                # Convert sklearns clustering result in sets of sample indices.
                new_current_clustering = [[] for _ in range(desired_num_clusters)]
                for sample_indices, cluster_index in zip(current_clustering, agglomerative_clustering.labels_):
                    new_current_clustering[cluster_index] += sample_indices
                current_clustering = new_current_clustering
                
                # Create a new clustering checkpoint by creating a new train-test split with new clustering.
                sample_indices_train, sample_indices_test, clustering_checkpoint_directory,\
                    checkpoints_directory, affinities_directory, mean_arithmetic_means =\
                        create_and_save_train_test_split(current_clustering, experiment_directory, hp)
                
                # Break out of the loop incase of early stopping
                del arithmetic_means
                break

            # Cleanup
            del arithmetic_means
            gc.collect()
            torch.cuda.empty_cache()

        # Cleanup
        del model
        gc.collect()
        torch.cuda.empty_cache()

In [4]:
run_experiment(hp)

Running experiment with hyperparameters:
{'dataset': 'imagenet-full', 'initial_clustering': {'path': 'results/deepcluster-10000/clustering.npy', 'minimum_initial_cluster_size': 80}, 'num_merges_per_network_update': 3000, 'desired_num_clusters': 1000, 'epochs': 15, 'affinity': {'batch_size': 128, 'train_ratio': 0.5, 'log_interval': 200}, 'device': device(type='cuda'), 'learning_rate': 0.001, 'weight_decay': 0, 'sobel': False, 'augmentation': {'random_horizontal_flip': False, 'random_resized_crop': False}}
Load initial clustering from results/deepcluster-10000/clustering.npy
Removed all clusters that have less than 80 observations. 1207/10000 (12.1%) clusters removed. 69888/1281167 (5.5%) observations removed.
Save train-test split to results/real/imagenet-full/8793/current_clustering.npy
Save mean arithmetic means to results/real/imagenet-full/8793/mean_arithmetic_means.csv
#Clusters: 8793
Epoch: 1 , mean arithmetic means: []
Batches processed: 200/4715 (4.2%), ETA: 38m:17s, Loss: 0.690

/home/chris/miniconda2/envs/pytorch4/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2555904 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))


Batches processed: 800/4715 (17.0%), ETA: 43m:53s, Loss: 0.679192
Batches processed: 1000/4715 (21.2%), ETA: 43m:34s, Loss: 0.632173
Batches processed: 1200/4715 (25.5%), ETA: 42m:44s, Loss: 0.652837
Batches processed: 1400/4715 (29.7%), ETA: 41m:07s, Loss: 0.640191


/home/chris/miniconda2/envs/pytorch4/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2555904 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))


Batches processed: 1600/4715 (33.9%), ETA: 39m:39s, Loss: 0.664033
Batches processed: 1800/4715 (38.2%), ETA: 37m:37s, Loss: 0.629190
Batches processed: 2000/4715 (42.4%), ETA: 35m:18s, Loss: 0.588241
Batches processed: 2200/4715 (46.7%), ETA: 33m:01s, Loss: 0.590340
Batches processed: 2400/4715 (50.9%), ETA: 30m:37s, Loss: 0.513809


...

Batches processed: 4400/4727 (93.1%), ETA: 03m:39s, Loss: 0.006538
Batches processed: 4600/4727 (97.3%), ETA: 01m:25s, Loss: 0.005285
Batches processed: 4727/4727 (100.0%), ETA: 0s, Loss: 0.004569
Save model and optimizer state to results/real/imagenet-full/2793/checkpoints/8.pth.tar
Batches processed: 200/4738 (4.2%), ETA: 46m:09s
Batches processed: 400/4738 (8.4%), ETA: 43m:52s


/home/chris/miniconda2/envs/pytorch4/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2555904 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))


Batches processed: 600/4738 (12.7%), ETA: 41m:51s
Batches processed: 800/4738 (16.9%), ETA: 39m:15s
Batches processed: 1000/4738 (21.1%), ETA: 37m:06s
Batches processed: 1200/4738 (25.3%), ETA: 35m:08s
Batches processed: 1400/4738 (29.5%), ETA: 33m:13s
Batches processed: 1600/4738 (33.8%), ETA: 31m:12s
Batches processed: 1800/4738 (38.0%), ETA: 29m:18s
Batches processed: 2000/4738 (42.2%), ETA: 27m:17s
Batches processed: 2200/4738 (46.4%), ETA: 25m:19s
Batches processed: 2400/4738 (50.7%), ETA: 23m:21s
Batches processed: 2600/4738 (54.9%), ETA: 21m:24s
Batches processed: 2800/4738 (59.1%), ETA: 19m:24s
Batches processed: 3000/4738 (63.3%), ETA: 17m:25s


/home/chris/miniconda2/envs/pytorch4/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2555904 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))


Batches processed: 3200/4738 (67.5%), ETA: 15m:23s
Batches processed: 3400/4738 (71.8%), ETA: 13m:24s
Batches processed: 3600/4738 (76.0%), ETA: 11m:23s
Batches processed: 3800/4738 (80.2%), ETA: 09m:23s
Batches processed: 4000/4738 (84.4%), ETA: 07m:22s
Batches processed: 4200/4738 (88.6%), ETA: 05m:22s
Batches processed: 4400/4738 (92.9%), ETA: 03m:22s
Batches processed: 4600/4738 (97.1%), ETA: 01m:23s
Batches processed: 4738/4738 (100.0%), ETA: 0s
Save mean arithmetic means to results/real/imagenet-full/2793/mean_arithmetic_means.csv
Epoch: 8, Train loss: 26.01, mean arithmetic mean: 0.9885675907
Save affinities to results/real/imagenet-full/2793/affinities/8.npy
Early stopping. Load previous affinities from results/real/imagenet-full/2793/affinities/7.npy
Merging from 2793 clusters to 1000 clusters.
Save train-test split to results/real/imagenet-full/1000/current_clustering.npy
Save mean arithmetic means to results/real/imagenet-full/1000/mean_arithmetic_means.csv
